In [1]:
import os
import librosa
import random
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
from sklearn.mixture import GaussianMixture
#import python_speech_features as mfcc
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from tqdm import tqdm
import joblib
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
import os
import numpy as np
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
from pathlib import Path
from IPython.display import display, Audio

In [3]:
#shutil.rmtree("/kaggle/working/speaker-recognition-dataset")

In [4]:
!cp -r "../input/speaker-recognition-dataset" ./

In [5]:
data_directory = "./speaker-recognition-dataset/16000_pcm_speeches"
audio_folder = "audio"
noise_folder = "noise"

audio_path = os.path.join(data_directory, audio_folder)
noise_path = os.path.join(data_directory, noise_folder)

In [6]:
for folder in os.listdir(data_directory):
    if os.path.isdir(os.path.join(data_directory, folder)):
        if folder in [audio_folder, noise_folder]:
            
            continue
        elif folder in ["other", "_background_noise_"]:
            
            shutil.move(
                os.path.join(data_directory, folder),
                os.path.join(noise_path, folder),
            )
        else:
            shutil.move(
                os.path.join(data_directory, folder),
                os.path.join(audio_path, folder),
            )

In [7]:
num_noise=[]
for i in os.listdir(noise_path):
    p=os.listdir(noise_path+"/"+i)
    len_class=len(p)
    num_noise.append(len_class)
num_noise

[2, 4]

In [8]:
noise_paths = []
for subdir in os.listdir(noise_path):
    subdir_path = Path(noise_path) / subdir
    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]

In [9]:
command = (
    "for dir in `ls -1 " + noise_path + "`; do "
    "for file in `ls -1 " + noise_path + "/$dir/*.wav`; do "
    "sample_rate=`ffprobe -hide_banner -loglevel panic -show_streams "
    "$file | grep sample_rate | cut -f2 -d=`; "
    "if [ $sample_rate -ne 16000 ]; then "
    "ffmpeg -hide_banner -loglevel panic -y "
    "-i $file -ar 16000 temp.wav; "
    "mv temp.wav $file; "
    "fi; done; done"
)
os.system(command)

0

In [10]:
SAMPLING_RATE = 16000

def load_noise_sample(path):
    # Load the audio file with librosa
    sample, sampling_rate = librosa.load(path, sr=None)

    if sampling_rate == SAMPLING_RATE:
        # Number of slices of 16000 each that can be generated from the noise sample
        slices = int(len(sample) / SAMPLING_RATE)
        
        # Reshape the array to create slices
        sample = np.reshape(sample[:slices * SAMPLING_RATE], (slices, SAMPLING_RATE))
        
        return sample
    else:
        print("Sampling rate for {} is incorrect. Ignoring it".format(path))
        return None

noises = []
for path in noise_paths:
    sample = load_noise_sample(path)
    if sample is not None:
        noises.extend(sample)

# Convert list to numpy array
noises = np.array(noises)

print(
    "{} noise files were split into {} noise samples where each is {} sec. long".format(
        len(noise_paths), len(noises), SAMPLING_RATE // SAMPLING_RATE
    )
)

6 noise files were split into 354 noise samples where each is 1 sec. long


In [11]:
noises

array([[ 0.6796875 ,  0.32202148,  0.41156006, ..., -0.07870483,
        -0.00308228,  0.1557312 ],
       [ 0.12347412, -0.0255127 , -0.06317139, ...,  0.03457642,
         0.2083435 ,  0.4513855 ],
       [ 0.42971802,  0.2899475 ,  0.40582275, ..., -0.59317017,
        -0.49490356, -0.60446167],
       ...,
       [ 0.04284668,  0.00843811, -0.00296021, ...,  0.03219604,
        -0.00515747, -0.03042603],
       [-0.04034424, -0.04818726, -0.05633545, ...,  0.00631714,
         0.03829956,  0.02713013],
       [-0.02148438, -0.05950928, -0.05187988, ..., -0.00717163,
        -0.01553345, -0.01739502]], dtype=float32)

In [12]:
def return_features(audio):
  y, sr = librosa.load(audio, sr=None)
  mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
  mfcc_delta = librosa.feature.delta(mfccs)
  mfcc_delta2 = librosa.feature.delta(mfccs, order=2)
  #zero_c = librosa.zero_crossings(mfccs, pad=False)
  #spectral_c = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
  #spectral_con = librosa.feature.spectral_contrast(y=y,sr=sr)
  features = np.vstack([mfccs, mfcc_delta, mfcc_delta2])
  #features_pca = pca.fit_transform(features)
  return (features.T)

In [13]:
class_label ={'Benjamin_Netanyau': 0, 'Jens_Stoltenberg': 1, 'Julia_Gillard': 2, 'Magaret_Tarcher': 3, 'Nelson_Mandela': 4}

In [14]:
path = '/kaggle/working/speaker-recognition-dataset/16000_pcm_speeches/audio'

In [15]:
os.listdir(path)

['Julia_Gillard',
 'Magaret_Tarcher',
 'Nelson_Mandela',
 'Benjamin_Netanyau',
 'Jens_Stoltenberg']

## Rename Files

In [16]:
for i in tqdm(class_label):
    folder_path = os.path.join(path,i)
    for file_name in os.listdir(folder_path):
        audio_file = os.path.join(folder_path,file_name)
        audio_name = i + '_' + file_name
        new_audio_name = os.path.join(folder_path,audio_name)
        #print(audio_file)
        os.rename(audio_file,new_audio_name)

100%|██████████| 5/5 [00:00<00:00, 21.25it/s]


In [17]:
file_list = []
class_list = []
for i in class_label:
    folder_path = os.path.join(path,i)
    for file_name in os.listdir(folder_path):
        file_list.append(file_name)
        class_list.append(i)
    

In [18]:
file_path='dataset.csv'
test_pred = pd.DataFrame({'Filename': file_list, 'TARGET': class_list})
test_pred.to_csv(file_path, mode='w', header=True, index=False)

In [19]:
len(file_list)

7501

In [20]:
data_set = pd.read_csv('dataset.csv')

In [21]:
data_set

,Filename,TARGET
0,Benjamin_Netanyau_1194.wav,Benjamin_Netanyau
1,Benjamin_Netanyau_260.wav,Benjamin_Netanyau
2,Benjamin_Netanyau_1032.wav,Benjamin_Netanyau
3,Benjamin_Netanyau_536.wav,Benjamin_Netanyau
4,Benjamin_Netanyau_855.wav,Benjamin_Netanyau
...,...,...
7496,Nelson_Mandela_1266.wav,Nelson_Mandela
7497,Nelson_Mandela_951.wav,Nelson_Mandela
7498,Nelson_Mandela_748.wav,Nelson_Mandela
7499,Nelson_Mandela_87.wav,Nelson_Mandela


In [22]:
data_set_train ,data_set_test = train_test_split(data_set,test_size=0.3,random_state=42)

In [23]:
data_set_train

,Filename,TARGET
4692,Magaret_Tarcher_1006.wav,Magaret_Tarcher
5263,Magaret_Tarcher_484.wav,Magaret_Tarcher
3277,Julia_Gillard_565.wav,Julia_Gillard
6400,Nelson_Mandela_1478.wav,Nelson_Mandela
2802,Jens_Stoltenberg_1218.wav,Jens_Stoltenberg
...,...,...
5191,Magaret_Tarcher_1438.wav,Magaret_Tarcher
5226,Magaret_Tarcher_1302.wav,Magaret_Tarcher
5390,Magaret_Tarcher_1248.wav,Magaret_Tarcher
860,Benjamin_Netanyau_533.wav,Benjamin_Netanyau


In [24]:
def return_features_with_noise(audio, noises):
    y, sr = librosa.load(audio, sr=None)
    
    # Select a random noise sample and ensure it's the same length as `y`
    noise = random.choice(noises)
    if len(noise) > len(y):
        noise = noise[:len(y)]
    else:
        noise = np.pad(noise, (0, max(0, len(y) - len(noise))), 'constant')

    # Add the noise to the audio
    y_noisy = y + noise*0.5

    # Extract MFCCs and their deltas as before
    mfccs = librosa.feature.mfcc(y=y_noisy, sr=sr, n_mfcc=13)
    mfcc_delta = librosa.feature.delta(mfccs)
    mfcc_delta2 = librosa.feature.delta(mfccs, order=2)

    features = np.vstack([mfccs, mfcc_delta, mfcc_delta2])
    return features.T

In [25]:
def extract_combined_features(audio_file, noises):
    # Extract features without noise
    features_clean = return_features(audio_file)

    # Extract features with noise
    features_noisy = return_features_with_noise(audio_file, noises)

    # Combine the features
    combined_features = np.concatenate((features_clean, features_noisy), axis=0)
    
    return combined_features



In [26]:
audio_features = []

# Loop over each speaker class
for speaker in tqdm(class_label):
    speaker_features = []

    # Folder path for the speaker
    folder_path = os.path.join(path, speaker)

    # Loop over each file for the speaker
    for j in data_set_train.index:
        if data_set_train['TARGET'][j] == speaker:
            audio_file_name = data_set_train['Filename'][j]
            audio_file = os.path.join(folder_path, audio_file_name)

            # Extract combined features
            combined_features = extract_combined_features(audio_file, noises)

            # Append the features for this file to the list
            speaker_features.extend(combined_features)

    audio_features.append(speaker_features)

100%|██████████| 5/5 [03:05<00:00, 37.14s/it]


In [27]:
noises

array([[ 0.6796875 ,  0.32202148,  0.41156006, ..., -0.07870483,
        -0.00308228,  0.1557312 ],
       [ 0.12347412, -0.0255127 , -0.06317139, ...,  0.03457642,
         0.2083435 ,  0.4513855 ],
       [ 0.42971802,  0.2899475 ,  0.40582275, ..., -0.59317017,
        -0.49490356, -0.60446167],
       ...,
       [ 0.04284668,  0.00843811, -0.00296021, ...,  0.03219604,
        -0.00515747, -0.03042603],
       [-0.04034424, -0.04818726, -0.05633545, ...,  0.00631714,
         0.03829956,  0.02713013],
       [-0.02148438, -0.05950928, -0.05187988, ..., -0.00717163,
        -0.01553345, -0.01739502]], dtype=float32)

In [28]:
for i in range(5):
    print(len(audio_features[i]))

65408
67328
68800
68544
65920


In [29]:
def train_gmm(data):
  gm = GaussianMixture(n_components=4, max_iter=160)
  gm.fit(data)
  return gm

In [30]:
gmm_model=[]
i=0
for temp in tqdm(audio_features):
  gmm_model.append(train_gmm(temp))
  #input('wait')
  #print(i)
  i=i+1

100%|██████████| 5/5 [00:59<00:00, 11.81s/it]


In [31]:
for i in tqdm(range(5)):
    filename = 'gmm'+str(i)+'.joblib'
    joblib.dump(gmm_model[i],filename)
    print(i)

100%|██████████| 5/5 [00:00<00:00, 495.25it/s]

0
1
2
3
4


In [32]:
saved_gmm=[]

for i in range(5):
    filename = 'gmm'+str(i)+'.joblib'
    load_gmm = joblib.load(filename)
    saved_gmm.append(load_gmm)

In [33]:
saved_gmm


[GaussianMixture(max_iter=160, n_components=4),
 GaussianMixture(max_iter=160, n_components=4),
 GaussianMixture(max_iter=160, n_components=4),
 GaussianMixture(max_iter=160, n_components=4),
 GaussianMixture(max_iter=160, n_components=4)]

In [34]:
def get_label(test_data,gmm_model):
  likelihood=[]
  test_mfcc = return_features(test_data)
  for class_model in gmm_model:
    likelihood.append(class_model.score(test_mfcc))
    #print(likelihood)
    # print(likelihood.index(max(likelihood)))
  return likelihood.index(max(likelihood))

In [35]:
label_final= []
for i in tqdm(data_set_test.index):
    audio_file = data_set_test['Filename'][i]
    folder_name = data_set_test['TARGET'][i]
    file_path = os.path.join(path,folder_name,audio_file)
    label_final.append(get_label(file_path,gmm_model))
    

100%|██████████| 2251/2251 [00:54<00:00, 41.35it/s]


In [36]:
#label_final

In [37]:
data_set_test

,Filename,TARGET
2310,Jens_Stoltenberg_385.wav,Jens_Stoltenberg
3347,Julia_Gillard_1426.wav,Julia_Gillard
3322,Julia_Gillard_468.wav,Julia_Gillard
4550,Magaret_Tarcher_76.wav,Magaret_Tarcher
5049,Magaret_Tarcher_1042.wav,Magaret_Tarcher
...,...,...
1773,Jens_Stoltenberg_351.wav,Jens_Stoltenberg
5476,Magaret_Tarcher_1448.wav,Magaret_Tarcher
1209,Benjamin_Netanyau_100.wav,Benjamin_Netanyau
3937,Julia_Gillard_571.wav,Julia_Gillard


In [38]:
mapped_list = []
for i in label_final:
    for key,value in class_label.items():
        if(i==value):
            mapped_list.append(key)
        #if(i == j.value):
        #    mapped_list.append(j.key)

In [39]:
count = Counter(mapped_list)

In [40]:
count

Counter({'Benjamin_Netanyau': 476,
         'Nelson_Mandela': 470,
         'Jens_Stoltenberg': 453,
         'Magaret_Tarcher': 429,
         'Julia_Gillard': 423})

In [41]:
accuracy = accuracy_score(data_set_test['TARGET'],mapped_list)

In [42]:
print(accuracy)

0.9968902709906708


EER

In [43]:
def get_scores(test_data, gmm_models):
    test_mfcc = return_features(test_data)
    scores = [model.score(test_mfcc) for model in gmm_models]
    return scores

In [44]:
from sklearn.metrics import roc_curve
from scipy.optimize import brentq
from scipy.interpolate import interp1d

def calculate_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)
    return eer, thresh

In [45]:
y_true = []
y_scores = []

for index, row in data_set_test.iterrows():
    audio_file = row['Filename']
    true_label = row['TARGET']
    file_path = os.path.join(path, true_label, audio_file)

    if os.path.exists(file_path):
        scores = get_scores(file_path, saved_gmm)
        true_label_index = class_label[true_label]
        y_scores.append(scores[true_label_index])
        y_true.append(1)  # Genuine case

        # Add impostor cases
        for impostor_label in class_label:
            if impostor_label != true_label:
                impostor_label_index = class_label[impostor_label]
                y_scores.append(scores[impostor_label_index])
                y_true.append(0)  # Impostor case

In [46]:
eer, threshold = calculate_eer(y_true, y_scores)
print(f"EER: {eer}, Threshold: {threshold}")

EER: 0.23722789871168407, Threshold: -108.82020017408117


FOR SCRIPT FILE

In [47]:
gmm_models = [joblib.load(f'gmm{i}.joblib') for i in range(5)]

In [48]:
def get_speaker_label(feature, gmm_models):
    """Determine the most likely speaker for the given feature."""
    scores = [model.score(feature) for model in gmm_models]
    return np.argmax(scores)

In [49]:
label_to_speaker = {value: key for key, value in class_label.items()}

In [50]:
def compare_speakers(file1, file2, gmm_models, label_to_speaker):
    """Compare two speakers and return True if they are the same, along with their names."""
    features1 = return_features(file1)
    features2 = return_features(file2)
    
    speaker1_label = get_speaker_label(features1, gmm_models)
    speaker2_label = get_speaker_label(features2, gmm_models)

    speaker1_name = label_to_speaker[speaker1_label]
    speaker2_name = label_to_speaker[speaker2_label]
    
    same_speaker = speaker1_label == speaker2_label
    return same_speaker, speaker1_name, speaker2_name

In [51]:
with open('/kaggle/input/testpair/test_pairs.txt', 'r') as file:
    pairs = [line.strip().split(',') for line in file.readlines()]
for file1, file2 in pairs:
    same_speaker, speaker1, speaker2 = compare_speakers(file1, file2, gmm_models, label_to_speaker)
    print("speaker 1 :",speaker1)
    print("speaker 2 :",speaker2)
    print("Same speaker :",same_speaker)

speaker 1 : Benjamin_Netanyau
speaker 2 : Julia_Gillard
Same speaker : False
speaker 1 : Nelson_Mandela
speaker 2 : Nelson_Mandela
Same speaker : True


In [52]:
file1="/kaggle/working/speaker-recognition-dataset/16000_pcm_speeches/audio/Benjamin_Netanyau/Benjamin_Netanyau_560.wav"
file2="/kaggle/working/speaker-recognition-dataset/16000_pcm_speeches/audio/Julia_Gillard/Julia_Gillard_29.wav"

In [53]:
same_speaker, speaker1, speaker2 = compare_speakers(file1, file2, gmm_models, label_to_speaker)
print("speaker 1 :",speaker1)
print("speaker 2 :",speaker2)
print("Same speaker :",same_speaker)

speaker 1 : Benjamin_Netanyau
speaker 2 : Julia_Gillard
Same speaker : False


In [54]:
file3="/kaggle/working/speaker-recognition-dataset/16000_pcm_speeches/audio/Julia_Gillard/Julia_Gillard_707.wav"
file4="/kaggle/working/speaker-recognition-dataset/16000_pcm_speeches/audio/Julia_Gillard/Julia_Gillard_0.wav"

In [55]:
same_speaker, speaker1, speaker2 = compare_speakers(file3, file4, gmm_models, label_to_speaker)
print("speaker 1 :",speaker1)
print("speaker 2 :",speaker2)
print("Same speaker :",same_speaker)

speaker 1 : Julia_Gillard
speaker 2 : Jens_Stoltenberg
Same speaker : False


In [56]:
file5="/kaggle/input/noisy-data/benjamin.wav"
file6="/kaggle/working/speaker-recognition-dataset/16000_pcm_speeches/audio/Benjamin_Netanyau/Benjamin_Netanyau_1400.wav"

In [57]:
same_speaker, speaker1, speaker2 = compare_speakers(file5, file6, gmm_models, label_to_speaker)
print("speaker 1 :",speaker1)
print("speaker 2 :",speaker2)
print("Same speaker :",same_speaker)

speaker 1 : Benjamin_Netanyau
speaker 2 : Benjamin_Netanyau
Same speaker : True
